a. import the package needed & define the deivce used

In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

b. mounting the drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


c.open file & train the sentence piece encoder

In [ ]:
with open("/content/drive/MyDrive/files/dataset.txt" , "r" , encoding="utf-8") as file:
  text = file.read()

import sentencepiece as spm
spm.SentencePieceTrainer.train(input='/content/drive/MyDrive/files/dataset.txt', model_prefix='m', vocab_size=3000) #setting vocab to higher value to encode sub words
sp = spm.SentencePieceProcessor(model_file='/content/m.model')

d. Load the dataset and encode it

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/files/dataset.csv")
df.columns=["question" , "answer"]

In [ ]:
def encode_data():
  x = [torch.tensor(sp.encode(i)) for i in df["question"]]
  y = [torch.tensor(sp.encode(i)) for i in df["answer"]]
  padding_x  = pad_sequence(x , batch_first=True , padding_value=0)
  padding_y  = pad_sequence(y , batch_first=True , padding_value=0)

  desired_padding = padding_x.size(1) - padding_y.size(1)
  padding_y =  F.pad(padding_y , (0, desired_padding), value=0)
  return padding_x , padding_y

X,Y = encode_data()

e. Variable Initialization

In [ ]:
batch_size = 4 #number of inputs processed in the same time
block_size = 307 #length of the block
num_iter = (len(X)-100)//batch_size
learning_rate = 1e-3
n_embd = 64 # size of the embeddings for example each token represented with 64 embedding
n_head = 4 # number of heads that works in parallel in multi headed
num_layers = 5
eval_interval = 100 #print losess each 100 iteration
eval_iters = 200
vocab_size = 3000
torch.manual_seed(1337)

f. Generate Batch

In [ ]:
trainX = X[:-100]
validX = X[-100:]
trainY  =Y[:-100]
validY = Y[-100:]
position = 0
def get_batch(split):
  if split == "train":
    x = torch.stack([trainX[i] for i in range(position,position+batch_size)])
    y =  torch.stack([trainY[i] for i in range(position,position+batch_size)])
  else :
    ix = torch.randint(100, (batch_size,))
    x = torch.stack([validX[i] for i in ix])
    y =  torch.stack([validY[i] for i in ix])
  x, y = x.to(device), y.to(device)
  return x , y

get_batch("valid")

(tensor([[119,  10,  25,  ...,   0,   0,   0],
         [ 61,  12,   7,  ...,   0,   0,   0],
         [ 25,  10, 561,  ...,   0,   0,   0],
         [ 61,   3,  21,  ...,   0,   0,   0]], device='cuda:0'),
 tensor([[ 481, 1563,  124,  ...,    0,    0,    0],
         [  47,   12,   70,  ...,    0,    0,    0],
         [ 380,  128,    8,  ...,    0,    0,    0],
         [  50,  177,  600,  ...,    0,    0,    0]], device='cuda:0'))

g. Transformer Modules

h. Testing

In [ ]:
context = "hello how you doing ?"
converted = [sp.encode(context)]
converted = torch.tensor(converted , device=device)
converted =  F.pad(converted , (0, 307 - len(converted) ), value=0)
print(converted)
print(sp.decode(m.generate(converted, max_number_tokens=20)[0].tolist()))

tensor([[107,  48, 103, 124,  10, 338,   3,  11,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0